## 完整的模型训练（以CIFAR-10数据集为例）
 1. 准备数据集
2. 利用DataLoader加载数据
3. 搭建神经网络
4. 损失函数
5. 优化器
6. 设置训练网络的一些参数
7. 训练网络
8. 保存模型

可以用cuda()将模型和数据放到GPU上加速训练

一般更多时候使用 **.to(device)** 将模型和数据放到GPU上加速训练

只有 *数据（输入数据和目标数据）和损失函数* 才能放到GPU上加速训练，模型和优化器不能放到GPU上加速训练





In [6]:
import torch
#准备数据集
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from model import *

train_data = torchvision.datasets.CIFAR10(root='./dataset', train=True, transform=torchvision.transforms.ToTensor(), download=True)
test_data = torchvision.datasets.CIFAR10(root='./dataset', train=False, transform=torchvision.transforms.ToTensor(), download=True)

train_data_size = len(train_data)
test_data_size = len(test_data)

print("训练数据集的长度为：{}".format(train_data_size))
print("测试数据集的长度为：{}".format(test_data_size))

Files already downloaded and verified
Files already downloaded and verified
训练数据集的长度为：50000
测试数据集的长度为：10000


In [8]:
# 利用DataLoader加载数据
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# 搭建神经网络
class Wisdom(nn.Module):
    def __init__(self):
        super(Wisdom, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(64*4*4, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)

In [10]:
# 创建网络模型（一般都从同一个文件夹下的python文件导入）

#定义训练的设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

wisdom = Wisdom()
wisdom = wisdom.to(device)
#wisdom = wisdom.cuda()

# 损失函数
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)
#loss_fn = loss_fn.cuda()

#优化器
learning_rate = 0.01
optimizer = torch.optim.SGD(wisdom.parameters(), lr=learning_rate)

# 设置训练网络的一些参数
total_train_step = 0 # 训练的次数
total_test_step = 0 # 测试的次数
epoch = 10 # 训练的轮数

writer = SummaryWriter("../logs_train")
for i in range(epoch):
    print("--------第{}轮训练--------".format(i+1))
    # 训练步骤开始
    wisdom.train() # 将模型设置为训练模式,这个函数会把模型中的BN和Dropout等层设置为训练模式,会改变权重
    for data in train_dataloader:
        imgs, targets = data
        imgs = imgs.to(device)
        targets = targets.to(device)
        # imgs = imgs.cuda()
        # targets = targets.cuda()
        outputs = wisdom(imgs)
        loss = loss_fn(outputs, targets)

        # 反向传播 + 优化器优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_step += 1
        if total_train_step % 100 == 0:
            print("训练次数：{}, Loss:{}".format(total_train_step, loss.item()))
            writer.add_scalar("train_loss", loss.item(), total_train_step)
    # 测试步骤开始
    wisdom.eval() # 将模型设置为测试模式,这个函数会把模型中的BN和Dropout等层设置为测试模式,不会改变权重
    total_test_loss = 0
    total_accuracy = 0
    with torch.no_grad(): # 不需要计算梯度
        for data in test_dataloader:
            imgs, targets = data
            imgs = imgs.to(device)
            targets = targets.to(device
            # imgs = imgs.cuda()
            # targets = targets.cuda()
            outputs = wisdom(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            accuracy = (outputs.argmax(1) == targets).sum()
            #argmax()返回最大值的索引 1表示按行取最大值,0表示按列取最大值
            total_accuracy += accuracy
    print("整体测试集上的Loss：{}".format(total_test_loss))
    print("整体测试集上的准确率：{}".format(total_accuracy / test_data_size))
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    total_test_step += 1

    torch.save(wisdom, "wisdom_{}.pth".format(i))
    print("模型已保存")
writer.close()

--------第1轮训练--------
训练次数：100, Loss:2.2875001430511475
训练次数：200, Loss:2.284886598587036
训练次数：300, Loss:2.2530033588409424
训练次数：400, Loss:2.164703845977783
训练次数：500, Loss:2.042515277862549
训练次数：600, Loss:2.029047727584839
训练次数：700, Loss:1.9855668544769287
整体测试集上的Loss：313.3469811677933
整体测试集上的准确率：0.28459998965263367
模型已保存
--------第2轮训练--------
训练次数：800, Loss:1.8656933307647705
训练次数：900, Loss:1.852642297744751
训练次数：1000, Loss:1.9685132503509521
训练次数：1100, Loss:1.972744107246399
训练次数：1200, Loss:1.7248820066452026
训练次数：1300, Loss:1.693921685218811
训练次数：1400, Loss:1.7424116134643555
训练次数：1500, Loss:1.7876776456832886
整体测试集上的Loss：300.74436247348785
整体测试集上的准确率：0.314300000667572
模型已保存
--------第3轮训练--------
训练次数：1600, Loss:1.7611724138259888
训练次数：1700, Loss:1.6243528127670288
训练次数：1800, Loss:1.8906781673431396
训练次数：1900, Loss:1.7407386302947998
训练次数：2000, Loss:1.9418278932571411
训练次数：2100, Loss:1.5108281373977661
训练次数：2200, Loss:1.483242392539978
训练次数：2300, Loss:1.7552871704101562
整体测试集上的Loss：2